In [1]:
import os
import sys
from typing import Tuple
from datetime import datetime
from pathlib import Path


def recursive_root(path: str, find="sumo-uc-23"):
    if os.path.split(path)[-1] == find:
        return Path(path)
    return recursive_root(os.path.split(path)[0], find=find)


ROOT = recursive_root(os.path.abspath("."))
sys.path.append(str(ROOT))


In [2]:
import numpy as np
import pandas as pd
import utm
import polars as pl

from shapely.geometry import Point, Polygon, LineString
from src.data_analysis import AnalysisRadar
from innosent_api.utils.approach_info import ApproachInfo


## Create the Radar Objects

In [3]:
radar_info = {
    # "Radar 137": {
    #     "KML": ROOT / "geodata" / "Radar137.kml",
    # },
    # "Radar 136": {
    #     "KML": ROOT / "geodata" / "Radar136.kml",
    # },
    "Radar 142": {
        "KML": ROOT / "geodata" / "Radar142.kml",
    },
    "Radar 141": {
        "KML": ROOT / "geodata" / "Radar141.kml",
    },
}


In [4]:
radars = [
    AnalysisRadar(str(r_info[1]["KML"]), r_info[0]) for r_info in radar_info.items()
]

radars = {
    radar.name: radar for radar in radars
}

Radar 142
INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.
Radar 141
INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


## Read in the Polars Data

In [5]:
radars_df = pl.read_parquet(
    "/Volumes/BittleLab/7 DOE Project/radar-data/prepared_data/2_6_23_ALL_Radars.parquet"
)

# there are some duplicate rows in the data. drop them and take the first one
radars_df = radars_df.groupby(
    by=["radar", "ui32_objectID", "time"]
).first()

radars_df = radars_df.sort("time")

def get_df_from_polars(radar: AnalysisRadar, pl_df: pl.DataFrame) -> pd.DataFrame:
    ip_address = f"10.160.7.{radar.name.split('radar')[-1].strip()}"
    df = pl_df.filter(pl.col("radar") == ip_address)

    return df.to_pandas()

## Label the Boxes

In [6]:
date = radars_df.select('time_cst').to_pandas().iloc[0].dt.date.values[0]
save_path = ROOT / "data" / date.strftime("%Y-%m-%d") 
save_path.mkdir(parents=True, exist_ok=True)

In [8]:
for r_name, radar in radars.items():
        radar.df = get_df_from_polars(radar, radars_df)
        # convert the x/y to lat/lon
        radar.radar_xy_2_latlon()
        # convert to rw time
        # radar.apply_rw_time()
        radar.df["dt"] = radar.df.pop("time_cst")
        # label boxes
        radar.label_boxes()
        # save the data
        radar.df.to_parquet(save_path / f"{r_name}.parquet")


### Find Vehicles that Cross the Stop Bar of the Boxes

In [9]:
def check_line_string_intersection(x, box: ApproachInfo):
    # check if the line string intersects the box
    if len(x) < 2:
        return False
    line = LineString(x[["f32_positionX_m", "f32_positionY_m"]].values)
    return line.intersects(box.stop_bar)


In [10]:
from tqdm import tqdm

data = []
for radar in radars.values():
    for veh, veh_df in tqdm(
        radar.df.groupby("ui32_objectID"), total=radar.df.ui32_objectID.nunique()
    ):
        # check if the line string intersects the box
        for box in veh_df.box.unique():
            if box != "":
                approach = [a for n, a in radar.approaches if n == box][0]
                if check_line_string_intersection(veh_df, approach):
                    data.append(
                        {
                            "veh": veh,
                            "box": box,
                            "radar": radar.name,
                            "cross_time": veh_df["dt"].iloc[
                                np.linalg.norm(
                                    veh_df[
                                        ["f32_positionX_m", "f32_positionY_m"]
                                    ].values
                                    - approach.end_point,
                                    axis=1,
                                ).argmin()
                            ],
                        }
                    )


100%|██████████| 26564/26564 [00:05<00:00, 5074.99it/s]


In [11]:
count_df = pd.DataFrame(data)

In [12]:
# save the count data
count_df.to_parquet(save_path / "count_data.parquet")